In [10]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import operator
from openml import datasets, tasks, flows, config

import itertools

def powerset(L):
    pset = set()
    for n in xrange(len(L) + 1):
        for sset in itertools.combinations(L, n):
            pset.add(sset)
    return pset


In [11]:
class Edge:
    def __init__(self, source, dest, f, t):
        self.source = source
        self.dest = dest
        self.f = f
        self.t = t
graph = {'edges':
            [Edge(0,1,3,1), Edge(0,2, 3, 1), Edge(1,3,3,25), 
             Edge(3,4,3,0), Edge(2,5, 2,25), Edge(2,4, 3,0),
             Edge(4, 6, 3, 20), Edge(6,7,1,60), Edge(6,8, 2,30)],
        'vertices':
             {0:10., 1:8., 2:2., 3:40., 4:42., 5:1., 6:30., 7:2., 8: 3.0}}

In [12]:
def o_neighbors(graph, v):
    n = []
    for e in graph['edges']:
        if e.source == v:
            n = n + [e.dest]
    return n
def get_edge(graph, v0, v1):
    for e in graph['edges']:
        if e.source == v0 and e.dest == v1:
            return e
    return None

def path(graph, v0, v1):
    def find_path(graph, v0, v1, path = []):
        path = path + [v0]
        active = o_neighbors(graph,v0)
        if v0 == v1:
            return [path]
        paths  = []
        for v in active:
            if v not in path:
                newpaths = find_path(graph, v, v1, path)
                for newpath in newpaths:
                    paths.append(newpath)

        return paths
    all_paths = find_path(graph, v0, v1)
    return set().union(*all_paths)  
def path_edge(graph, v0, v1):
    vs = path(graph, v0, v1)
    edges = []
    for p in powerset(vs):
        if len(p) == 2:
            e = get_edge(graph, p[0], p[1])
            if e:
                edges.append(e)
    return edges
             
# for e in path_edge(graph, 0, 7):
#     print e.source, e.dest

In [3]:
def total_size(graph, mat):
    total = 0
    for k,v in graph['vertices'].iteritems():
        if k in mat:
            total = total + v
    return total

In [4]:
def recreation_cost(graph, v):
    cost = 0
    for e in path_edge(graph, 0, v):
        if e.dest != v:
            cost+= (e.t)
        else:
            alpha = e.f
            cost += (e.t)
    return alpha * cost
#recreation_cost(graph, 7)

In [5]:
def total_recreation_cost(graph, mat):
    total = 0
    for e in graph['edges']:
        if e.dest not in mat:
            total = total + (e.f * e.t)
    return total

In [6]:
import itertools
def powerset(L):
    pset = set()
    for n in xrange(len(L) + 1):
        for sset in itertools.combinations(L, n):
            pset.add(sset)
    return pset

def brute_force(graph, MAX_SIZE = 55):
    results = {}
    for l in powerset(graph['vertices'].keys()):
        if 0 in l:
            weight = 0
            for v in l:
                weight += graph['vertices'][v]
            results[l] = (total_recreation_cost(graph, l), weight)
    feasible = {}
    for k,v in results.iteritems():
        if v[1] <= MAX_SIZE:
            feasible [k] = v
    return list(sorted(feasible.items(), key=lambda v:v[1][0])[0][0])

result = brute_force(graph)  
print sorted(result), total_size(graph, result), total_recreation_cost(graph, result)

[0, 1, 5, 6, 7, 8] 54.0 78


In [18]:
def rho(graph, v):
    cost = 0
    for e in path_edge(graph, 0, v):
        if e.dest != v:
            print e.t
            cost+= (e.t)
        else:
            alpha = e.f
            cost += (e.t)    
    return alpha * cost

def algorithm_in_paper(graph,  MAX_SIZE = 55):
    mat = [0]
    T = graph['vertices'][0]
    estimates = {}
    for i in range(1,len(graph['vertices'])):
         estimates[i] = rho(graph, i) / float(graph['vertices'][i])
    for k,v in sorted(estimates.items(), key=operator.itemgetter(1), reverse=True):
        print k, graph['vertices'][k], estimates[k], T
        if T + graph['vertices'][k] <= MAX_SIZE:
            mat.append(k)
            T = T + graph['vertices'][k]
    print mat, T, total_recreation_cost(graph, mat)


In [16]:
algorithm_in_paper(graph)

7 2.0 53.5 10.0
5 1.0 52.0 12.0
8 3.0 51.3333333333 13.0
6 30.0 4.7 16.0
3 40.0 1.95 46.0
4 42.0 1.92857142857 46.0
2 2.0 1.5 46.0
1 8.0 0.375 48.0
[0, 7, 5, 8, 6, 2] 48.0 78


In [19]:
rho(graph, 7)

1
20
1
25
0
0


107